<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/highlight_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

In [ ]:
!pip install --target=$my_path transformers
!apt-get update
!apt-get g++ openjdk-8-jdk 
!pip3 install --target=$my_path konlpy
!pip install --target=$my_path soykeyword
!pip install --target=$my_path krwordrank
!pip install --target=$my_path kss

In [3]:
import kss
from konlpy.tag import Hannanum, Okt
import re
import time
import numpy as np
import heapq
import pandas as pd
from operator import itemgetter
from collections import deque
from ast import literal_eval
from collections import defaultdict
from soykeyword.lasso import LassoKeywordExtractor
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
%cd /content/mnt/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
from frameBERT import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
h = Hannanum()
okt = Okt()

/content/mnt/Shared drives/BigDATA TEAM 10/OpenInformationExtraction

###DEVICE: cpu


In [4]:
text = '''
평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. 인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까? 그리고 그 근거는 무엇인가? 
일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대칙 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 대어난 능력과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. 평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데 목표를 두고 있다. ‘이유 없는 차별 금지’라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별저 대우를 하는 것을 허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근거에 대한 물음이다.
근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다. 그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런하려고 한다. 그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는 형식저 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이 더 높은 도덕적 권위를 지닌 다른 이넘과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다.
롤스는 인간 평등의 근거를 설명하면서 영역 성질 (range property) 개념을 도입한다. 예를 들어 어떤 원의 내부에 있는 점들은 그 위치가 서로 다르지만 원의 내부에 있다는 점에서 동일한 영역 성질을 갖는다. 반면에 원의 내부에 있는 집과 원의 외부에 였는 점은원의 경계선을 기준으로 서로 다른 영역 성질을 갖는다. 그는 평등한 대우를 받기 위한 영역 성질로서 ‘도덕적 인격'을 제시한다. 도덕적 인격이란 도덕적 호소가 가능하고 그런 호소에 관심을 기이는 능력이 있다는 것인데, 이 능력을 최소치만 갖고 있다면 평등한 대우에 대한 권한을 갖게 된다. 도덕적 인격이라고 해서 도덕적으로 훌륭하다는 뜻이 아니라 도덕과 무관하다는 말과 대비되는 뜻으로 쓰고 있다. 그런데 어린 아이는 인격체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다. 이에 대해 롤스는 도덕적 인격을 규정하는 최소한의 요구 조건은 잠재적 능력이지 그것의 실현 여부가 아니기에 어린 아이도 평등한 존재라고 말한다. 싱어는 위와 같은 롤스의 시도를 비판한다. 도덕에 대한 민감성의 수준은 사람에 따라 다르다. 그래서 도덕적 인격의 능력이 그렇게 중요하다면 그것을 갖춘 정도에 따라 도덕적 위계를 다르게 하지 말아야 할 이유가 분명하지 않다고 말한다. 
'''

In [8]:
class Text:
  def __init__(self, text):
    self.text = text
    text = re.sub('\n',' ',text)
    text = re.sub("'",' ',text)
    paragraphs = text.split('\n')
    self.paragraphs = [i for i in paragraphs if i]
    self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]

  def keywords(self, num = 40):
    wordrank_extractor = KRWordRank(min_count=4, max_length=10)
    keywords, rank, graph = wordrank_extractor.extract(self.paragraphs, num_keywords=num)
    p=[]
    kw=[]
    for k, v in keywords.items():
      p.append(okt.pos(k))
    for ls in p:
      for tup in ls:
          if tup[1].startswith("N"):
            kw.append(tup[0])
    return kw
    
class Highlight(Text):
  def __init__(self, text, candidates=None):
    self.text = text
    self.newtext = deepcopy(self.text)
    self.cand = candidates
    self.candidates = candidates
    if self.cand==None:
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만'
        conj = conj.replace("'","")
        conj = conj.replace(", ",",")
        self.candidates = conj.split(",")
        self.idx = [(i, i+len(candidate)) for candidate in self.candidates for i in findall(candidate, self.text)]
    else:
        self.candidates = candidates
        self.idx = [(i, i+len(candidate)) for candidate in self.candidates for i in findall(candidate, self.newtext)]

  def findall(p, s):
      i = s.find(p)
      while i != -1:
          yield i
          i = s.find(p, i+1)

  def add_tags(self):
    for i in range(len(self.idx)):
        idx = [(start, start+len(candidate)) for candidate in self.candidates for start in self.findall(candidate, self.newtext)]
        
        word = self.newtext[idx[i][0]:idx[i][1]]
        print(word)
        
        if self.cand==None:
          tagged = "<mark style='background-color: #0F4C81'>%s</mark>" % (word)
          print(tagged)
        else:
          tagged = "<mark style='background-color: #FF6F61'>%s</mark>" % (word)
          print(tagged)
        self.newtext = tagged.join([self.newtext[:idx[i][0]], self.newtext[idx[i][1]:]])

    return self.newtext


In [9]:
def main():
  t = Text(text)
  kw=t.keywords()
  print(kw)
  h = Highlight(text, candidates = None)
  h = Highlight(h.add_tags(), kw)
  css_format = h.add_tags()
  print(css_format)
  

In [10]:
main()

['도덕', '평등', '인간', '이유', '영역', '모든', '대우', '롤스', '규칙', '원', '인격', '성질', '능력', '대한', '근거', '정의', '제시', '무엇', '자연', '사람']


NameError: ignored


<mark style='background-color: #FF6F61'>평등</mark>은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. <mark style='background-color: #FF6F61'>인간</mark>은 가령 인종이나 성별과 상관없이 누구나 <mark style='background-color: #FF6F61'>평등</mark>하다고 생각한다. <mark style='background-color: #FF6F61'>모든</mark> <mark style='background-color: #FF6F61'>인간</mark>은 <mark style='background-color: #FF6F61'>평등</mark>하다고 말하는데, 이 말은 무슨 뜻일까? <mark style='background-color: #0F4C81'>그리고</mark> 그 <mark style='background-color: #FF6F61'>근거</mark>는 <mark style='background-color: #FF6F61'>무엇</mark>인가? 
일<mark style='background-color: #0F4C81'>단</mark> 이 말을 <mark style='background-color: #FF6F61'>모든</mark> <mark style='background-color: #FF6F61'>인간</mark>을 <mark style='background-color: #FF6F61'>모든</mark> 측면에서 똑같이 <mark style='background-color: #FF6F61'>대우</mark>하는 절대칙 <mark style='background-color: #FF6F61'>평등</mark>으로 생각하는 이는 없다. <mark style='background-color: #FF6F61'>인간</mark>은 저마다 다르게 가지고 대어난 <mark style='background-color: #FF6F61'>능력</mark>과 소질을 똑같게 만들 수 없기 때문이다. 절대적 <mark style='background-color: #FF6F61'>평등</mark>은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. <mark style='background-color: #FF6F61'>평등</mark>에 <mark style='background-color: #FF6F61'>대한</mark> 요구는 <mark style='background-color: #FF6F61'>모든</mark> 불<mark style='background-color: #FF6F61'>평등</mark>을 악으로 보는 것이 아니라 충분한 <mark style='background-color: #FF6F61'>이유</mark>가 <mark style='background-color: #FF6F61'>제시</mark>되지 않은 불<mark style='background-color: #FF6F61'>평등</mark>을 제거하는 데 목표를 두고 있다. ‘<mark style='background-color: #FF6F61'>이유</mark> 없는 차별 금지’라는 조건적 <mark style='background-color: #FF6F61'>평등</mark> <mark style='background-color: #FF6F61'>원</mark>칙은 차별 <mark style='background-color: #FF6F61'>대우</mark>를 할 때는 <mark style='background-color: #FF6F61'>이유</mark>를 <mark style='background-color: #FF6F61'>제시</mark>할 것을 요구하고 있다. 이것은 어떤 <mark style='background-color: #FF6F61'>이유</mark>가 <mark style='background-color: #FF6F61'>제시</mark>된다면 특정한 부류에 속하는 <mark style='background-color: #FF6F61'>사람</mark>들에게는 <mark style='background-color: #FF6F61'>평등</mark>한 <mark style='background-color: #FF6F61'>대우</mark>를, 그 부류에 속하지 않는 <mark style='background-color: #FF6F61'>사람</mark>들에게는 차별저 <mark style='background-color: #FF6F61'>대우</mark>를 하는 것을 허용한다. 그렇다면 <mark style='background-color: #FF6F61'>사람</mark>들을 특정한 부류로 구분하는 기준은 <mark style='background-color: #FF6F61'>무엇</mark>인가? 이것은 바로 <mark style='background-color: #FF6F61'>평등</mark>의 <mark style='background-color: #FF6F61'>근거</mark>에 <mark style='background-color: #FF6F61'>대한</mark> 물음이다.
근대의 여러 인권 선언에 나타난 <mark style='background-color: #FF6F61'>평등</mark> 개념은 개인들 사이의 <mark style='background-color: #FF6F61'>평등</mark>성을 타고난 <mark style='background-color: #FF6F61'>자연</mark>적 권리로 간주하였다. <mark style='background-color: #0F4C81'>하지만</mark> 이러한 <mark style='background-color: #FF6F61'>자연</mark>권 이론은 <mark style='background-color: #FF6F61'>무엇</mark>이 <mark style='background-color: #FF6F61'>자연</mark>적 권리이고 권리의 존재가 자명한 <mark style='background-color: #FF6F61'>이유</mark>가 <mark style='background-color: #FF6F61'>무엇</mark>인지 등의 문제에 부딪히게 된다. <mark style='background-color: #0F4C81'>그래서</mark> <mark style='background-color: #FF6F61'>롤스</mark>는 기존의 <mark style='background-color: #FF6F61'>자연</mark>권 사상에 의존하지 않는 방시으로 <mark style='background-color: #FF6F61'>인간</mark> <mark style='background-color: #FF6F61'>평등</mark>의 <mark style='background-color: #FF6F61'>근거</mark>를 마런하려고 한다. 그는 어떤 <mark style='background-color: #FF6F61'>규칙</mark>이 공평하고 일관되게 운영되며, 그 <mark style='background-color: #FF6F61'>규칙</mark>에 따라 유사한 경우는 유사하게 취급된다면 형식적 <mark style='background-color: #FF6F61'>정의</mark>는 실현된다고 본다. <mark style='background-color: #0F4C81'>하지만</mark> <mark style='background-color: #FF6F61'>롤스</mark>는 형식저 <mark style='background-color: #FF6F61'>정의</mark>에 따라 <mark style='background-color: #FF6F61'>규칙</mark>을 준수하는 것만으로는 <mark style='background-color: #FF6F61'>정의</mark>를 담보할 수 없다고 생각한다. 그 <mark style='background-color: #FF6F61'>규칙</mark>이 더 높은 <mark style='background-color: #FF6F61'>도덕</mark>적 권위를 지닌 다른 이넘과 충돌할 수 었기에, 실질적 <mark style='background-color: #FF6F61'>정의</mark>가 보장되기 위해서는 <mark style='background-color: #FF6F61'>규칙</mark>의 내용이 중요한 것이다.
<mark style='background-color: #FF6F61'>롤스</mark>는 <mark style='background-color: #FF6F61'>인간</mark> <mark style='background-color: #FF6F61'>평등</mark>의 <mark style='background-color: #FF6F61'>근거</mark>를 설명하면서 <mark style='background-color: #FF6F61'>영역</mark> <mark style='background-color: #FF6F61'>성질</mark> (range property) 개념을 도입한다. <mark style='background-color: #0F4C81'>예를 들어</mark> 어떤 <mark style='background-color: #FF6F61'>원</mark>의 내부에 있는 점들은 그 위치가 서로 다르지만 <mark style='background-color: #FF6F61'>원</mark>의 내부에 있다는 점에서 동일한 <mark style='background-color: #FF6F61'>영역</mark> <mark style='background-color: #FF6F61'>성질</mark>을 갖는다. <mark style='background-color: #0F4C81'>반면에</mark> <mark style='background-color: #FF6F61'>원</mark>의 내부에 있는 집과 <mark style='background-color: #FF6F61'>원</mark>의 외부에 였는 점은<mark style='background-color: #FF6F61'>원</mark>의 경계선을 기준으로 서로 다른 <mark style='background-color: #FF6F61'>영역</mark> <mark style='background-color: #FF6F61'>성질</mark>을 갖는다. 그는 <mark style='background-color: #FF6F61'>평등</mark>한 <mark style='background-color: #FF6F61'>대우</mark>를 받기 위한 <mark style='background-color: #FF6F61'>영역</mark> <mark style='background-color: #FF6F61'>성질</mark>로서 ‘<mark style='background-color: #FF6F61'>도덕</mark>적 <mark style='background-color: #FF6F61'>인격</mark>'을 <mark style='background-color: #FF6F61'>제시</mark>한다. <mark style='background-color: #FF6F61'>도덕</mark>적 <mark style='background-color: #FF6F61'>인격</mark>이란 <mark style='background-color: #FF6F61'>도덕</mark>적 호소가 가능하고 그런 호소에 관심을 기이는 <mark style='background-color: #FF6F61'>능력</mark>이 있다는 것인데, 이 <mark style='background-color: #FF6F61'>능력</mark>을 최소치만 갖고 있다면 <mark style='background-color: #FF6F61'>평등</mark>한 <mark style='background-color: #FF6F61'>대우</mark>에 <mark style='background-color: #FF6F61'>대한</mark> 권한을 갖게 된다. <mark style='background-color: #FF6F61'>도덕</mark>적 <mark style='background-color: #FF6F61'>인격</mark>이라고 해서 <mark style='background-color: #FF6F61'>도덕</mark>적으로 훌륭하다는 뜻이 아니라 <mark style='background-color: #FF6F61'>도덕</mark>과 무관하다는 말과 대비되는 뜻으로 쓰고 있다. <mark style='background-color: #0F4C81'>그런데</mark> 어린 아이는 <mark style='background-color: #FF6F61'>인격</mark>체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다. 이에 대해 <mark style='background-color: #FF6F61'>롤스</mark>는 <mark style='background-color: #FF6F61'>도덕</mark>적 <mark style='background-color: #FF6F61'>인격</mark>을 규정하는 최소한의 요구 조건은 잠재적 <mark style='background-color: #FF6F61'>능력</mark>이지 그것의 실현 여부가 아니기에 어린 아이도 <mark style='background-color: #FF6F61'>평등</mark>한 존재라고 말한다. 싱어는 위와 같은 <mark style='background-color: #FF6F61'>롤스</mark>의 시도를 비판한다. <mark style='background-color: #FF6F61'>도덕</mark>에 <mark style='background-color: #FF6F61'>대한</mark> 민감성의 수준은 <mark style='background-color: #FF6F61'>사람</mark>에 따라 다르다. <mark style='background-color: #0F4C81'>그래서</mark> <mark style='background-color: #FF6F61'>도덕</mark>적 <mark style='background-color: #FF6F61'>인격</mark>의 <mark style='background-color: #FF6F61'>능력</mark>이 그렇게 중요하다면 그것을 갖춘 정도에 따라 <mark style='background-color: #FF6F61'>도덕</mark>적 위계를 다르게 하지 말아야 할 <mark style='background-color: #FF6F61'>이유</mark>가 분명하지 않다고 말한다. 
